In [206]:
# from urllib.parse import quote_from_bytes, unquote_to_bytes
# import uuid
import numpy as np
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

def pkcs7(b, size=128, v=b''):
    bsz = len(b)
    size //= 8
    sz = size * (bsz // size + 1)
    pad = v if v else bytes([sz - bsz])
    return b + pad * (sz - bsz)

def cipher(plain, key=K):
    aes = Cipher(algorithms.AES128(key), modes.ECB()).encryptor()
    pad = padding.PKCS7(128).padder()
    return aes.update(pad.update(plain) + pad.finalize()) + aes.finalize()

def decipher(ciphered, key=K):
    aes = Cipher(algorithms.AES128(key), modes.ECB()).decryptor()
    upad = padding.PKCS7(128).unpadder()
    return upad.update(aes.update(ciphered) + aes.finalize()) + upad.finalize()

def cipher_blksz(cipher):
    p = b""
    e = cipher(p)
    s = len(e)
    offset = 0
    while s == len(e):
        p += b'A'
        e = cipher(p)
        offset += 1
    return len(e) - s, offset

In [80]:
K = np.random.bytes(16)

In [11]:
s = b"foo=bar&baz=qux&zap=zazzle"

In [422]:
def sanitize(s):
    """
    Proper sanitazing with urllib's quote_from_bytes makes this attack harder or impossible without compromises on padding values.

    Can't even try to insert \\0 charater in case the parser expects a null terminated string

    Might work if somehow the parser is reaaaally stupid and role=adminXXXXXXXXXXX gets accepted
    """
    # return quote_from_bytes(s)
    return s.replace(b'&', b'').replace(b'=', b'')

def decode_profile(s):
    return {
        k: v
        for k, v in [x.split(b'=') for x in s.split(b'&')]
    }

def encode_profile(p):
    return b"&".join([b"=".join([sanitize(i) for i in item]) for item in p.items()])

In [13]:
decode_profile(s)

{b'foo': b'bar', b'baz': b'qux', b'zap': b'zazzle'}

In [36]:
encode_profile(decode_profile(s))

b'foo=bar&baz=qux&zap=zazzle'

In [313]:
def profile_for(email):
    p = {
        b'email': email,
        b'uid': b'10',
        b'role': b'user'
    }
    return encode_profile(p)

In [407]:
p = profile_for(b'foo@bar.com')
print(p)
decode_profile(p)

b'email=foo@bar.com&uid=10&role=user'


{b'email': b'foo@bar.com', b'uid': b'10', b'role': b'user'}

In [72]:
def profile_oracle(email):
    return cipher(profile_for(email))

def decode_ciphered_profile(ciphered):
    return decode_profile(decipher(ciphered))

In [342]:
P = profile_oracle(b'foo@bar.com')

In [343]:
decode_ciphered_profile(P)

{b'email': b'foo@bar.com', b'uid': b'10', b'role': b'user'}

In [189]:
def get_prefix_size(cipher):
    blksz, _ = cipher_blksz(cipher)

    ini = cipher(b"")
    ini_blks = [ini[i:i+blksz] for i in range(0, len(ini), blksz)]
    p = b"A"
    e = cipher(p)
    e_blks = [e[i:i+blksz] for i in range(0, len(e), blksz)]

    # get initial potential full blocks
    i = 0
    while ini_blks[i] == e_blks[i]:
        i += 1
    psz = i * blksz

    pblk = ini_blks[i]
    nblk = e_blks[i]

    while pblk != nblk:
        p += b"A"
        e = cipher(p)
        e_blks = [e[i:i+blksz] for i in range(0, len(e), blksz)]
        pblk = nblk
        nblk = e_blks[i]
    
    return psz + blksz - len(p) + 1


def get_suffix_size(cipher):
    return len(cipher(b'')) - get_prefix_size(cipher) - cipher_blksz(cipher)[1]

In [205]:
def oracle_blk_enc(oracle, plain):
    """Computes the encrypted block 'cipher(plain)' with the cipher used in 'oracle'
    
    Args:
        oracle (bytes): Given an ECB with fixed unkown key that encrypts a plain text with a prefix and suffix of fixed sizes, encrypt 'plain'
        plain (bytes): bytes to be encrypted

    Returns:
        bytes: Encrypted block
    """
    blksz, _ = cipher_blksz(oracle)

    psz = get_prefix_size(oracle)

    prefix_offset = blksz - psz % blksz

    a = psz // blksz + blksz
    b = a + blksz

    return oracle(b'A' * prefix_offset + pkcs7(plain))[a:b]


In [207]:
oracle_blk_enc(profile_oracle, b'admin')

b'\x8c\xa4\x91\x025x<\xe2\xe0p\xbe\xac<\xefMM'

In [213]:
def align_last_blk_to(oracle, plain):
    """Find offset to align oracle's last block to 'cipher(plain)' (cipher is a cipher used by oracle)"""
    blk = oracle_blk_enc(oracle, plain)
    p = b''
    e = oracle(p)
    blksz, _ = cipher_blksz(oracle)
    while e[-blksz:] != blk:
        p += b'A'
        e = oracle(p)
    return len(p)


In [214]:
align_last_blk_to(profile_oracle, b'user')

15

In [314]:
def forge_admin_profile():
    blksz, _ = cipher_blksz(profile_oracle)
    admin_blk = oracle_blk_enc(profile_oracle, b'admin')
    N = align_last_blk_to(profile_oracle, b'user')
    return profile_oracle(b'A' * N)[:-blksz] + admin_blk

In [315]:
decode_ciphered_profile(forge_admin_profile())

{b'email': b'AAAAAAAAAAAAA', b'uid': b'10', b'role': b'admin'}